In [1]:
import h3
from shapely.geometry import Polygon
import folium
from folium.plugins import TimestampedGeoJson
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
import geopandas as gpd
import pandas as pd
import contextily as ctx
import imageio
import os

# Resolution 7 ~ ~1 km hex, resolution 8 ~ 200–300 m
RESOLUTION = 7
FILE_NAME = 'hokkaido.csv'

In [2]:
pokestops_jp = pd.read_csv(FILE_NAME)
pokestops_jp['h3_index'] = pokestops_jp.apply(
    lambda row: h3.latlng_to_cell(row['Fort_Latitude'], row['Fort_Longitude'], RESOLUTION),
    axis=1
)

PART 1 - BASIC TIMELINE ANIMATION

In [ ]:
# basic timeline animation

df = pokestops_jp.copy()
df['Timestamp'] = pd.to_datetime(df['Timestamp_local'])
# Convert each row into a GeoJSON feature
features = []
for _, row in df.iterrows():
    feature = {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': [row['Fort_Longitude'], row['Fort_Latitude']]
        },
        'properties': {
            'time': row['Timestamp'].isoformat(),
            'popup': f"PokéStop visited at {row['Timestamp']}",
            'icon': 'circle',
            'iconstyle':{
                'fillColor': 'red',
                'fillOpacity': 0.6,
                'stroke': 'false',
                'radius': 5
            }
        }
    }
    features.append(feature)

geojson = {
    'type': 'FeatureCollection',
    'features': features
}

In [ ]:
# center map
center_lat = df['Player_Latitude'].mean()
center_lon = df['Player_Longitude'].mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=11, tiles='CartoDB Voyager')

# visit https://python-visualization.github.io/folium/latest/user_guide/plugins/timestamped_geojson.html

TimestampedGeoJson(
    data=geojson,
    transition_time=100,  # ms between frames
    period='PT30M',        # time interval per frame
    add_last_point=True,   # keep last point visible
    auto_play=True,
    loop=False,
    max_speed=15,
    loop_button=True,
    date_options='YYYY/MM/DD HH:mm:ss',
    time_slider_drag_update=True
).add_to(m)

m.save("pokestop_timeline.html")
m

PART 2 - FINAL PROJECT: COMBINING HEX HEATMAP WITH TIMELINE ANIMATION

In [3]:
# data preparation
df = pokestops_jp.copy()

# convert timestamp object to datetime type
df['Timestamp_local'] = pd.to_datetime(df['Timestamp_local'])

# epoch seconds for animation
df['epoch'] = df['Timestamp_local'].astype('int64') // 10**9
df = df.sort_values('epoch')

# list to store the running total of spins per hex
cumulative_hex_counts = []
# dictionary to track cumulative spins per hex
hex_counter = {}

for _, row in df.iterrows():
    h = row['h3_index']
    hex_counter[h] = hex_counter.get(h, 0) + 1
    cumulative_hex_counts.append(hex_counter.copy())


In [15]:
# prepare geodataframe
gdf_points = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df['Fort_Longitude'], df['Fort_Latitude']),
    crs='EPSG:4326'
)

gdf_points = gdf_points.to_crs(epsg=3857)


# fixed map bounds
margin = 2000 # meters around points
minx, miny, maxx, maxy = gdf_points.total_bounds
xlim = (minx - margin, maxx + margin)
ylim = (miny - margin, maxy + margin)




In [17]:
# animation settings
output_dir = "frames_map"
os.makedirs(output_dir, exist_ok=True)
frames = []

cumulative_hex_info = {}  # hx -> {'count': total spins, 'forts': set()}

trail_length = 15  # number of previous points to show

In [18]:
# function to generate a gdf for each frame (row) 
def h3_to_gdf(hex_dict):
    geoms, alphas, counts = [], [], []
    for hx, info in hex_dict.items():
        coords = h3.cell_to_boundary(hx)
        poly = Polygon([(lon, lat) for lat, lon in coords])
        geoms.append(poly)
        # darken based on number of spins
        alphas.append(min(0.1 + 0.05*info['count'], 0.8))
        counts.append(info['unique_forts'])
    gdf = gpd.GeoDataFrame({'geometry': geoms, 'alpha': alphas, 'unique_forts': counts}, crs='EPSG:4326')
    return gdf.to_crs(epsg=3857)

In [20]:
# generate all the frames as png
for i, row in gdf_points.iterrows():
    fig, ax = plt.subplots(figsize=(8,8))
    
    # update cumulative hex info
    hx = df.loc[i, 'h3_index']
    cumulative_hex_info.setdefault(hx, {'count':0, 'forts':set()})
    cumulative_hex_info[hx]['count'] += 1
    cumulative_hex_info[hx]['forts'].add(row['fort_id'])  # make sure df has Fort_ID
    
    # prepare hex geodataframe
    hex_dict = {k:{'count':v['count'], 'unique_forts':len(v['forts'])} for k,v in cumulative_hex_info.items()}
    gdf_hex = h3_to_gdf(hex_dict)
    
    # plot hex
    gdf_hex.plot(ax=ax, color='red', alpha=gdf_hex['alpha'], edgecolor='grey')
    
    # annotate hex with number of unique forts
    for _, hx_row in gdf_hex.iterrows():
        x, y = hx_row.geometry.centroid.x, hx_row.geometry.centroid.y
        ax.text(x, y, str(hx_row['unique_forts']), ha='center', va='center', fontsize=8, color='black')
    
    # trail of raw points
    start_idx = max(0, i - trail_length)
    trail = gdf_points.iloc[start_idx:i+1]
    for j, tr in trail.iterrows():
        # alpha decreases faster for older points
        alpha = max(0.05, 0.8 * ((j - start_idx + 1)/len(trail))**2)
        ax.scatter(tr.geometry.x, tr.geometry.y, color='white', s=40, alpha=alpha)
    
    # add map tiles
    ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)
    
    # fixed map extents
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_aspect('equal')
    
    ax.axis('off')
    
    # save frame
    filename = f"{output_dir}/frame_{i:03d}.png"
    plt.savefig(filename, dpi=200, bbox_inches='tight',pad_inches=0)
    plt.close()
    frames.append(filename)

In [3]:
# export gif

# frames_dir = "frames_map"
# frames = sorted(
#     [os.path.join(frames_dir, f) for f in os.listdir(frames_dir) if f.endswith(".png")]
# )

images = [imageio.imread(f) for f in frames]
imageio.mimsave("pokemon_map_animation.gif", images, fps=4)